In [ ]:
# Módulos de Selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import selenium.webdriver.common.keys as Keys

# Otros módulos
import undetected_chromedriver as uc
import os
from dotenv import load_dotenv
import time
import random
from bs4 import BeautifulSoup as bs
import requests as req
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options
from zenrows import ZenRowsClient
from datetime import datetime

# Pandas para manejo de datos
import pandas as pd

In [ ]:
load_dotenv()

In [ ]:
URL = 'https://www.infojobs.net/'

In [ ]:
usuario = os.getenv('u_infojobs')
clave = os.getenv('c_infojobs')
empleos = ['analista de datos', 'cyber security', 'desarrollador web']

In [ ]:
driver = uc.Chrome(headless=False,use_subprocess=False)

In [ ]:
   # abre la web
driver.get(URL) # ingresa en la direccion de la URL 
# intenta encontrar las cookies y aceptar, si no encuentra impime ya tiene las cookies
try:
    driver.find_element(By.CSS_SELECTOR, '#didomi-notice-agree-button').click()
    print('Cookies aceptadas')
except Exception as e:
    print(e)
    print('Ya tienes la cookies')
#hacer click en acceso a candidatos
driver.find_element(By.CSS_SELECTOR, '#candidate_login').click()

# Esperar hasta que el boton exista o sea clickable
cuadro_usuario= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#email')))
# ingresar letra a letra usuario y clave
for letra in usuario:
    cuadro_usuario.send_keys(letra)
    time.sleep(random.uniform(0.3,0.75))
cuadro_clave= driver.find_element(By.CSS_SELECTOR, '#id-password')
for num in clave:
    cuadro_clave.send_keys(num)
    time.sleep(random.uniform(0.3,0.75))
cuadro_usuario.submit()

In [ ]:
lista_ofertas= []

for e in empleos:
    boton_empleos= driver.find_element(By.CSS_SELECTOR, '#nav-mobile > ul > li:nth-child(1) > a')
    driver.execute_script("arguments[0].click();", boton_empleos)
    time.sleep(5)
    # localizar cuadro de busqueda e ingresar el empleo y la ciudad para buscar
    cuadro_busqueda_oferta = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#searchBoxKeyword')))
    time.sleep(5)
    for letra in e:
        cuadro_busqueda_oferta.send_keys(letra)
        time.sleep(random.uniform(0.3,0.75))
    
    # darle click al boton buscar
    driver.find_element(By.CSS_SELECTOR, '#searchOffers').click()
    time.sleep(5)

    # hacer scrol a la pagina
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, 0);")
    alto_pagina = driver.execute_script("return document.body.scrollHeight;")
    driver.execute_script(f"window.scrollTo(0, {alto_pagina}/3);" )
    time.sleep(3)
    driver.execute_script(f"window.scrollTo(0, ({alto_pagina}*2)/3);" )
    time.sleep(3)
    driver.execute_script(f"window.scrollTo(0, {alto_pagina});" )
    time.sleep(5)
   
    # encontrar las ofertas por su clase
    elemento_selenium= driver.find_elements(By.CLASS_NAME,'ij-List-item')

    # encontrar la informacion en cada oferta
    descripcion= []
    ofertas= []
    for informacion in elemento_selenium:
        try:        
            html_elemento = informacion.get_attribute('outerHTML')
            soup = bs(html_elemento, 'html.parser')
            descripcion.append(soup)
        except Exception as ex:
            #print(f"Error catcheado {ex}")
            pass

    for info in descripcion:
        try:
            html= info.find('div', {'class': 'ij-OfferCardContent-description'})
            ofertas.append(html)
        except Exception as ex:
            #print(f"Error catcheado {ex}")
            pass

    for of in ofertas:
        if 'datos' in e:
            tipo_empleo= 'Analista de datos'
        elif 'cyber' in e:
            tipo_empleo= 'Cyber Security'
        elif 'web' in e: 
            tipo_empleo= 'Desarrollador Web'
        try:
            titulo = of.find('a').text
            link_oferta = of.find('a').attrs['href']
            empresa = of.find('h3').text
            link_empresa = of.find('h3').find('a').attrs['href']
            
            mas_info= of.find_all('li', {'class': 'ij-OfferCardContent-description-list-item'})
            if len(mas_info) >= 6:
                ciudad= mas_info[0].text
                modalidad= mas_info[1].text
                publicada= mas_info[2].text
                tipo_contrato= mas_info[3].text
                jornada= mas_info[4].text
                salario= mas_info[5].text
            else:
                ciudad= mas_info[0].text
                modalidad= 'No especifica'
                publicada= mas_info[1].text
                tipo_contrato= mas_info[2].text
                jornada= mas_info[3].text
                salario= mas_info[4].text 

            oferta_dict = {
                            'fecha_extraccion': datetime.now().date(),
                            'publicada': publicada,
                            'tipo_empleo': tipo_empleo,
                            'titulo': titulo,
                            'link_oferta': link_oferta,
                            'empresa': empresa,
                            'link_empresa': link_empresa,
                            'ubicacion': ciudad,
                            'tipo_contrato': tipo_contrato,
                            'modalidad': modalidad,
                            'jornada': jornada,
                            'salario': salario
                            }
            
            if oferta_dict not in lista_ofertas:
                lista_ofertas.append(oferta_dict)
        except Exception as ex:
            #print(f"Error catcheado {ex}")
            pass
        
    #inicializo en que pagina estoy 
    pagina= 1
    print(f'pagina {pagina} lista, del empleo {e}')
    try:
        boton_siguiente= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#app > div > div.ij-LayoutCandidate-body > div.ij-AdvertisingRoadblock > div.ij-AdvertisingRoadblockContainer > div.ij-AdvertisingRoadblockContainerChild > div > div > div.ij-ContentSearch-main > div.ij-ContentSearch-list > div.ij-ComponentPagination > ul > li:nth-child(6) > button')))
        time.sleep(5)
        driver.execute_script("arguments[0].click();", boton_siguiente)
        
        # encontrar las ofertas por su clase
        elemento_selenium= driver.find_elements(By.CLASS_NAME,'ij-List-item')

        # encontrar la informacion en cada oferta
        descripcion= []
        ofertas= []

        for informacion in elemento_selenium:
            try:        
                html_elemento = informacion.get_attribute('outerHTML')
                soup = bs(html_elemento, 'html.parser')
                descripcion.append(soup)
            except Exception as ex:
                #print(f"Error catcheado {ex}")
                pass

        for info in descripcion:
            try:
                html= info.find('div', {'class': 'ij-OfferCardContent-description'})
                ofertas.append(html)
            except Exception as ex:
                #print(f"Error catcheado {ex}")
                pass

        for of in ofertas:
            if 'datos' in e:
                tipo_empleo= 'Analista de datos'
            elif 'cyber' in e:
                tipo_empleo= 'Cyber Security'
            elif 'web' in e: 
                tipo_empleo= 'Desarrollador Web'
            try:
                titulo = of.find('a').text
                link_oferta = of.find('a').attrs['href']
                empresa = of.find('h3').text
                link_empresa = of.find('h3').find('a').attrs['href']
                
                mas_info= of.find_all('li', {'class': 'ij-OfferCardContent-description-list-item'})
                if len(mas_info) >= 6:
                    ciudad= mas_info[0].text
                    modalidad= mas_info[1].text
                    publicada= mas_info[2].text
                    tipo_contrato= mas_info[3].text
                    jornada= mas_info[4].text
                    salario= mas_info[5].text
                else:
                    ciudad= mas_info[0].text
                    modalidad= 'No especifica'
                    publicada= mas_info[1].text
                    tipo_contrato= mas_info[2].text
                    jornada= mas_info[3].text
                    salario= mas_info[4].text    
           
                oferta_dict = {
                                'fecha_extraccion': datetime.now().date(),
                                'publicada': publicada,
                                'tipo_empleo': tipo_empleo,
                                'titulo': titulo,
                                'link_oferta': link_oferta,
                                'empresa': empresa,
                                'link_empresa': link_empresa,
                                'ubicacion': ciudad,
                                'tipo_contrato': tipo_contrato,
                                'modalidad': modalidad,
                                'jornada': jornada,
                                'salario': salario
                                }
                if oferta_dict not in lista_ofertas:
                    lista_ofertas.append(oferta_dict)
            except Exception as ex:
                #print(f"Error catcheado {ex}")
                pass
        pagina+=1
        print(f'pagina {pagina} lista, del empleo {e}')

    except:
        try: 
            boton_mas_ofertas= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#app > div.ij-Box.ij-Container.ij-Container-wrapper.ij-Container-centered.ij-TemplateAdsPage > div > main > div.ij-Box.ij-MyInfojobs-preferenceOffers > div.ij-Box.ij-MyInfojobs-preferenceOffers-moreOffersButton.mt-l.mb-xl > a')))
            time.sleep(5)
            driver.execute_script("arguments[0].click();", boton_mas_ofertas)
            # encontrar las ofertas por su clase
            elemento_selenium= driver.find_elements(By.CLASS_NAME,'ij-List-item')
            time.sleep(5)

            # encontrar la informacion en cada oferta
            descripcion= []
            ofertas= []

            for informacion in elemento_selenium:
                try:        
                    html_elemento = informacion.get_attribute('outerHTML')
                    soup = bs(html_elemento, 'html.parser')
                    descripcion.append(soup)
                except Exception as ex:
                    #print(f"Error catcheado {ex}")
                    pass

            for info in descripcion:
                try:
                    html= info.find('div', {'class': 'ij-OfferCardContent-description'})
                    ofertas.append(html)
                except Exception as ex:
                    #print(f"Error catcheado {ex}")
                    pass

            for of in ofertas:
                if 'datos' in e:
                    tipo_empleo= 'Analista de datos'
                elif 'cyber' in e:
                    tipo_empleo= 'Cyber Security'
                elif 'web' in e: 
                    tipo_empleo= 'Desarrollador Web'
                try:
                    titulo = of.find('a').text
                    link_oferta = of.find('a').attrs['href']
                    empresa = of.find('h3').text
                    link_empresa = of.find('h3').find('a').attrs['href']
                    
                    mas_info= of.find_all('li', {'class': 'ij-OfferCardContent-description-list-item'})
                    if len(mas_info) >= 6:
                        ciudad= mas_info[0].text
                        modalidad= mas_info[1].text
                        publicada= mas_info[2].text
                        tipo_contrato= mas_info[3].text
                        jornada= mas_info[4].text
                        salario= mas_info[5].text
                    else:
                        ciudad= mas_info[0].text
                        modalidad= 'No especifica'
                        publicada= mas_info[1].text
                        tipo_contrato= mas_info[2].text
                        jornada= mas_info[3].text
                        salario= mas_info[4].text 

                    oferta_dict = {
                                    'fecha_extraccion': datetime.now().date(),
                                    'publicada': publicada,
                                    'tipo_empleo': tipo_empleo,
                                    'titulo': titulo,
                                    'link_oferta': link_oferta,
                                    'empresa': empresa,
                                    'link_empresa': link_empresa,
                                    'ubicacion': ciudad,
                                    'tipo_contrato': tipo_contrato,
                                    'modalidad': modalidad,
                                    'jornada': jornada,
                                    'salario': salario
                                    }
                    
                    if oferta_dict not in lista_ofertas:
                        lista_ofertas.append(oferta_dict)
                  
                except Exception as ex:
                    #print(f"Error catcheado {ex}")
                    pass
            pagina+=1
                
        except:
            print('no hay mas ofertas')


    while pagina >= 2:
        try:
            # esperar hasta que se pueda clickar el boton
            boton_siguiente= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#app > div > div.ij-LayoutCandidate-body > div.ij-AdvertisingRoadblock > div.ij-AdvertisingRoadblockContainer > div.ij-AdvertisingRoadblockContainerChild > div > div > div.ij-ContentSearch-main > div.ij-ContentSearch-list > div.ij-ComponentPagination > ul > li:nth-child(7) > button')))
            time.sleep(5)
            driver.execute_script("arguments[0].click();", boton_siguiente)
           

            # scroll a toda la pagina
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.execute_script("window.scrollTo(0, 0);")
            alto_pagina = driver.execute_script("return document.body.scrollHeight;")
            driver.execute_script(f"window.scrollTo(0, {alto_pagina}/3);" )
            time.sleep(3)
            driver.execute_script(f"window.scrollTo(0, ({alto_pagina}*2)/3);" )
            time.sleep(3)
            driver.execute_script(f"window.scrollTo(0, {alto_pagina});" )
            time.sleep(5)

            # encontrar las ofertas por su clase
            elemento_selenium= driver.find_elements(By.CLASS_NAME,'ij-List-item')

            # encontrar la informacion en cada oferta
            descripcion= []
            ofertas= []

            for informacion in elemento_selenium:
                try:        
                    html_elemento = informacion.get_attribute('outerHTML')
                    soup = bs(html_elemento, 'html.parser')
                    descripcion.append(soup)
                except Exception as ex:
                    print(f"Error catcheado {ex}")

            for info in descripcion:
                try:
                    html= info.find('div', {'class': 'ij-OfferCardContent-description'})
                    ofertas.append(html)
                except Exception as ex:
                    print(f"Error catcheado {ex}")

            for of in ofertas:
                if 'datos' in e:
                    tipo_empleo= 'Analista de datos'
                elif 'cyber' in e:
                    tipo_empleo= 'Cyber Security'
                elif 'web' in e: 
                    tipo_empleo= 'Desarrollador Web'
                try:
                    titulo = of.find('a').text
                    link_oferta = of.find('a').attrs['href']
                    empresa = of.find('h3').text
                    link_empresa = of.find('h3').find('a').attrs['href']
                    
                    mas_info= of.find_all('li', {'class': 'ij-OfferCardContent-description-list-item'})
                    if len(mas_info) >= 6:
                        ciudad= mas_info[0].text
                        modalidad= mas_info[1].text
                        publicada= mas_info[2].text
                        tipo_contrato= mas_info[3].text
                        jornada= mas_info[4].text
                        salario= mas_info[5].text
                    else:
                        ciudad= mas_info[0].text
                        modalidad= 'No especifica'
                        publicada= mas_info[1].text
                        tipo_contrato= mas_info[2].text
                        jornada= mas_info[3].text
                        salario= mas_info[4].text 

                    oferta_dict = {
                                    'fecha_extraccion': datetime.now().date(),
                                    'publicada': publicada,
                                    'tipo_empleo': tipo_empleo,
                                    'titulo': titulo,
                                    'link_oferta': link_oferta,
                                    'empresa': empresa,
                                    'link_empresa': link_empresa,
                                    'ubicacion': ciudad,
                                    'tipo_contrato': tipo_contrato,
                                    'modalidad': modalidad,
                                    'jornada': jornada,
                                    'salario': salario
                                    }
                    
                    if oferta_dict not in lista_ofertas:
                        lista_ofertas.append(oferta_dict)
                except Exception as ex:
                    print(f"Error catcheado {ex}")
            pagina +=1
            print(f'pagina {pagina} lista, del empleo {e}')
        except Exception as ex:
            print(f"Error catcheado {ex}")
            print('No hay mas boton siguiente')
            break
driver.quit()
data= pd.DataFrame(lista_ofertas)  

In [ ]:
data.to_csv('data2_infojobs.csv', index=False)

In [ ]:
data.info()